In [1]:
import random
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
import warnings
warnings.filterwarnings("ignore")
from modelcaller import ModelCaller, MCconfig, mc_wrapd, mc_wrap
random.seed(42)

def generate_data(f, count=1000, scale=100):
    global globalx
    inputs = np.zeros((count, 3))
    outputs = np.zeros(count)
    for i in range(count):
        globalx = random.random() * scale
        x0 = random.random() * scale
        x1 = random.random() * scale
        inputs[i] = [x0, x1, globalx]
        outputs[i] = f(x0, x1)
    return inputs, outputs

def repeat_function(f, arity=2, count=10, scale=100):
    global globalx
    for _ in range(count):
        globalx = random.random() * scale
        args = [random.random() * scale for _ in range(arity)]
        f(*args)

In [2]:
@mc_wrapd(cargs=['globalx'])
def f(x0, x1): 
    global globalx
    return 3 * x0 + x1 + globalx

In [3]:
mc = f._mc
mc.print('A new wrapped MC with one context argument', full=True)

A new wrapped MC with one context argument: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, qlty_threshold=0.95, ncargs=0, call_target:host, #functions:0, model-qualities:[], #tdata:0, #edata:0; tinputs:[]; toutputs:[]; einputs:[]; eoutputs:[]


In [4]:
repeat_function(f)
mc.print('After a few function calls')

After a few function calls: call_target:host, #functions:0, model-qualities:[], #tdata:7, #edata:3; tinputs:...[[82.9, 61.9, 55.2], [10.1, 27.8, 23.3]]; toutputs:...[365.9, 81.4]; einputs:...[[22.0, 58.9, 54.5], [4.6, 22.8, 70.5]]; eoutputs:...[179.6, 107.0]


In [5]:
mc.add_model(LinearRegression())
mc.print('After training and evaluating the added model')

After adding a model: call_target:host, #functions:0, model-qualities:[False], #tdata:7, #edata:3; tinputs:...[[82.9, 61.9, 55.2], [10.1, 27.8, 23.3]]; toutputs:...[365.9, 81.4]; einputs:...[[22.0, 58.9, 54.5], [4.6, 22.8, 70.5]]; eoutputs:...[179.6, 107.0]
Compare Model 0 score = 1.0 with qlty_threshold 0.95 => quality = True
After training and evaluating the added model: call_target:both, #functions:0, model-qualities:[True], #tdata:7, #edata:3; tinputs:...[[82.9, 61.9, 55.2], [10.1, 27.8, 23.3]]; toutputs:...[365.9, 81.4]; einputs:...[[22.0, 58.9, 54.5], [4.6, 22.8, 70.5]]; eoutputs:...[179.6, 107.0]


In [6]:
repeat_function(f)
mc.print('After a few more function calls')

After a few more function calls: call_target:both, #functions:0, model-qualities:[True], #tdata:13, #edata:7; tinputs:...[[26.3, 58.5, 56.1], [99.6, 52.9, 38.2]]; toutputs:...[193.4, 389.9]; einputs:...[[99.8, 51.0, 21.9], [62.7, 79.2, 11.0]]; eoutputs:...[372.1, 278.4]


In [7]:
if mc.get_call_target() == 'both': 
    mc.merge_host()
    mc.print('After merging host function')
    repeat_function(f)
    mc.print('After a few more function calls')

After merging host function: call_target:MC, #functions:1, model-qualities:[True], #tdata:13, #edata:7; tinputs:...[[26.3, 58.5, 56.1], [99.6, 52.9, 38.2]]; toutputs:...[193.4, 389.9]; einputs:...[[99.8, 51.0, 21.9], [62.7, 79.2, 11.0]]; eoutputs:...[372.1, 278.4]
After a few more function calls: call_target:MC, #functions:1, model-qualities:[True], #tdata:18, #edata:12; tinputs:...[[55.0, 26.5, 47.5], [53.9, 73.0, 21.2]]; toutputs:...[239.0, 256.0]; einputs:...[[5.8, 87.8, 30.8], [6.9, 76.1, 48.6]]; eoutputs:...[135.9, 145.4]


In [8]:
midx = mc.add_model(MLPRegressor(hidden_layer_sizes=(), activation='identity'))
mc.print('After training and evaluating the added model')
repeat_function(f)
mc.print('After a few more function calls')

After adding a model: call_target:MC, #functions:1, model-qualities:[True, False], #tdata:18, #edata:12; tinputs:...[[55.0, 26.5, 47.5], [53.9, 73.0, 21.2]]; toutputs:...[239.0, 256.0]; einputs:...[[5.8, 87.8, 30.8], [6.9, 76.1, 48.6]]; eoutputs:...[135.9, 145.4]
Compare Model 1 score = -12.326023911638964 with qlty_threshold 0.95 => quality = False
After training and evaluating the added model: call_target:MC, #functions:1, model-qualities:[True, False], #tdata:18, #edata:12; tinputs:...[[55.0, 26.5, 47.5], [53.9, 73.0, 21.2]]; toutputs:...[239.0, 256.0]; einputs:...[[5.8, 87.8, 30.8], [6.9, 76.1, 48.6]]; eoutputs:...[135.9, 145.4]
After a few more function calls: call_target:MC, #functions:1, model-qualities:[True, False], #tdata:25, #edata:15; tinputs:...[[27.9, 25.0, 42.2], [55.0, 5.1, 86.1]]; toutputs:...[150.7, 256.3]; einputs:...[[33.8, 58.8, 22.5], [23.8, 66.9, 7.1]]; eoutputs:...[182.7, 145.4]


In [9]:
if mc.get_call_target() == 'MC':
    xy = generate_data(mc.get_host())
    mc.add_dataset(xy[0], xy[1])
    mc.print('After adding more data but before training')
    mc.train_all()
    mc.print('After training and evaluating with the new data')
    repeat_function(f)
    mc.print('After a few more function calls')

After adding more data but before training: call_target:MC, #functions:1, model-qualities:[True, False], #tdata:1025, #edata:15; tinputs:...[[38.7, 16.3, 28.5], [96.5, 85.7, 57.2]]; toutputs:...[160.8, 432.4]; einputs:...[[33.8, 58.8, 22.5], [23.8, 66.9, 7.1]]; eoutputs:...[182.7, 145.4]
Compare Model 0 score = 1.0 with qlty_threshold 0.95 => quality = True
Compare Model 1 score = -1.1893491694541063 with qlty_threshold 0.95 => quality = False
After training and evaluating with the new data: call_target:MC, #functions:1, model-qualities:[True, False], #tdata:1025, #edata:15; tinputs:...[[38.7, 16.3, 28.5], [96.5, 85.7, 57.2]]; toutputs:...[160.8, 432.4]; einputs:...[[33.8, 58.8, 22.5], [23.8, 66.9, 7.1]]; eoutputs:...[182.7, 145.4]
After a few more function calls: call_target:MC, #functions:1, model-qualities:[True, False], #tdata:1033, #edata:17; tinputs:...[[29.9, 15.0, 89.2], [80.2, 60.0, 80.5]]; toutputs:...[193.9, 381.3]; einputs:...[[67.8, 26.9, 64.7], [60.9, 29.8, 64.7]]; eoutpu

In [10]:
if mc.get_model_quality(midx) == False:
    mc.qlty_threshold = -100
    mc.print('After updating qlty_threshold', full=True)
    mc.eval_all()
    mc.print('After reevaluating all models with the new threshold')
    repeat_function(f)
    mc.print('After a few more function calls')

After updating qlty_threshold: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, qlty_threshold=-100, ncargs=1, call_target:MC, #functions:1, model-qualities:[True, False], #tdata:1033, #edata:17; tinputs:...[[29.9, 15.0, 89.2], [80.2, 60.0, 80.5]]; toutputs:...[193.9, 381.3]; einputs:...[[67.8, 26.9, 64.7], [60.9, 29.8, 64.7]]; eoutputs:...[295.0, 277.3]
Compare Model 0 score = 1.0 with qlty_threshold -100 => quality = True
Compare Model 1 score = -1.3934905891281475 with qlty_threshold -100 => quality = True
After reevaluating all models with the new threshold: call_target:MC, #functions:1, model-qualities:[True, True], #tdata:1033, #edata:17; tinputs:...[[29.9, 15.0, 89.2], [80.2, 60.0, 80.5]]; toutputs:...[193.9, 381.3]; einputs:...[[67.8, 26.9, 64.7], [60.9, 29.8, 64.7]]; eoutputs:...[295.0, 277.3]
After a few more function calls: call_target:MC, #functions:1, model-qualities:[True, True], #tdata:1041, #edata:19; tinputs:..

In [11]:
mc.remove_model(1)
mc.qlty_threshold = 0.95
mc.print('After removing the second model and reverting the threshold', full=True)

After removing the second model and reverting the threshold: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, qlty_threshold=0.95, ncargs=1, call_target:MC, #functions:1, model-qualities:[True], #tdata:1041, #edata:19; tinputs:...[[94.4, 7.1, 58.4], [38.2, 29.8, 86.7]]; toutputs:...[283.6, 210.9]; einputs:...[[3.5, 70.8, 38.8], [18.3, 68.0, 76.3]]; eoutputs:...[112.7, 186.5]


In [12]:
fidx = mc.add_function(lambda x: x * x)
mc.print('After adding a new function')

After adding a new function: call_target:MC, #functions:2, model-qualities:[True], #tdata:1041, #edata:19; tinputs:...[[94.4, 7.1, 58.4], [38.2, 29.8, 86.7]]; toutputs:...[283.6, 210.9]; einputs:...[[3.5, 70.8, 38.8], [18.3, 68.0, 76.3]]; eoutputs:...[112.7, 186.5]


In [13]:
mc.remove_function(fidx)
mc.print('After removing the last function')

After removing the last function: call_target:MC, #functions:1, model-qualities:[True], #tdata:1041, #edata:19; tinputs:...[[94.4, 7.1, 58.4], [38.2, 29.8, 86.7]]; toutputs:...[283.6, 210.9]; einputs:...[[3.5, 70.8, 38.8], [18.3, 68.0, 76.3]]; eoutputs:...[112.7, 186.5]


In [14]:
mc.remove_dataset()
mc.print('After removing all training data')
repeat_function(f)
mc.print('After a few more function calls')

After removing all training data: call_target:MC, #functions:1, model-qualities:[True], #tdata:0, #edata:19; tinputs:[]; toutputs:[]; einputs:...[[3.5, 70.8, 38.8], [18.3, 68.0, 76.3]]; eoutputs:...[112.7, 186.5]
After a few more function calls: call_target:MC, #functions:1, model-qualities:[True], #tdata:7, #edata:22; tinputs:...[[69.4, 34.4, 24.9], [59.6, 33.8, 72.8]]; toutputs:...[267.4, 285.5]; einputs:...[[3.8, 19.9, 38.3], [16.7, 82.4, 58.9]]; eoutputs:...[69.6, 191.4]


In [15]:
@mc.wrap_sensor()
def fcopy(x0, x1, x3):  # y
    return 3 * x0 + x1 + x3

In [16]:
repeat_function(fcopy, arity=3)
mc.print('After a few direct-sensor calls')

After a few direct-sensor calls: call_target:MC, #functions:1, model-qualities:[True], #tdata:12, #edata:27; tinputs:...[[63.0, 30.1, 66.2], [26.3, 52.3, 20.2]]; toutputs:...[285.4, 151.4]; einputs:...[[60.6, 13.7, 83.1], [11.8, 11.4, 10.6]]; eoutputs:...[278.5, 57.3]


In [17]:
@mc.wrap_sensor('inverse')
def finv(y, x1, x2):  # x1
    return (y - x1 -  x2) / 3

repeat_function(finv, arity=3)
mc.print('After a few inverse-sensor calls')

After a few inverse-sensor calls: call_target:MC, #functions:1, model-qualities:[True], #tdata:19, #edata:30; tinputs:...[[-11.7, 87.8, 9.3], [-27.8, 96.2, 27.2]]; toutputs:...[62.2, 40.1]; einputs:...[[-3.8, 17.5, 3.9], [-21.5, 64.9, 79.7]]; eoutputs:...[10.0, 80.0]


In [18]:
globalx = 1
y = f(2, 3)
y.callback(100.0)
for kind in ('tdata', 'edata'):
    idx, out = mc.find_data([2, 3, 1], kind)
    if idx >= 0:
        print(f"Feedback callback: {y:.1f} updated to {out} in _{kind}['outputs'][{idx}] for inputs [2, 3, 1]")

Feedback callback: 10.0 updated to 100.0 in _edata['outputs'][30] for inputs [2, 3, 1]


In [19]:
repeat_function(mc, arity=3)
mc.print('After a few MC calls')

After a few MC calls: call_target:MC, #functions:1, model-qualities:[True], #tdata:24, #edata:36; tinputs:...[[97.8, 67.0, 33.8], [66.2, 24.9, 34.6]]; toutputs:...[405.5, 245.4]; einputs:...[[5.0, 22.3, 64.3], [80.1, 44.8, 85.7]]; eoutputs:...[118.0, 351.0]


In [20]:
globalx = 1
y = f(2, 3)
mc.remove_dataset('tdata')
mc.remove_dataset('edata')
y.callback(100.0)

In [21]:
mc1 = ModelCaller(MCconfig(_ncargs=1))
mc1.print('A new unwrapped MC with one context argument', full=True)
mc1.add_model(mc.get_model(0), quality=True) # reuse model
repeat_function(mc1, arity=3)
mc1.print('After a few mc calls')

A new unwrapped MC with one context argument: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, qlty_threshold=0.95, ncargs=1, call_target:MC, #functions:0, model-qualities:[], #tdata:0, #edata:0; tinputs:[]; toutputs:[]; einputs:[]; eoutputs:[]
After adding a model: call_target:MC, #functions:0, model-qualities:[True], #tdata:0, #edata:0; tinputs:[]; toutputs:[]; einputs:[]; eoutputs:[]
After a few mc calls: call_target:MC, #functions:0, model-qualities:[True], #tdata:8, #edata:2; tinputs:...[[45.1, 65.8, 95.6], [4.9, 93.5, 83.9]]; toutputs:...[296.6, 192.0]; einputs:[[65.7, 71.6, 11.4], [16.4, 14.0, 71.7]]; eoutputs:[280.1, 135.0]


In [22]:
@mc_wrapd(auto_id=None)
def f2(x0, x1):  # y
    return 3 * x0 + x1
f2(10,11)
f2._mc.print('A new wrapped MC with only auto-id and no other context argument, after a function call', full=True)

A new wrapped MC with only auto-id and no other context argument, after a function call: auto_cache=False, auto_id=True, auto_eval=False, auto_train=False, edata_fraction=0.3, feedback_fraction=0, qlty_threshold=0.95, ncargs=1, call_target:host, #functions:0, model-qualities:[], #tdata:0, #edata:0; tinputs:[]; toutputs:[]; einputs:[]; eoutputs:[]


In [23]:
m = LinearRegression()
m.fit([[1, 2, 3], [3, 4, 5]], [9, 10])
fpredict = mc_wrap(m.predict)  # wrapping a predefined function
fpredict([[10, 20, 30]])
fpredict._mc.print('A new MC, after wrapping a model.predict and calling MC', full=True)

A new MC, after wrapping a model.predict and calling MC: auto_cache=True, auto_id=False, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, qlty_threshold=0.95, ncargs=0, call_target:host, #functions:0, model-qualities:[], #tdata:1, #edata:0; tinputs:[[10, 20, 30]]; toutputs:[[18.0]]; einputs:[]; eoutputs:[]


In [24]:
m2 = LinearRegression()
m = mc_wrap(m, kind='model', auto_id=True)
mc2 = m._mc
mc2.merge_host()
mc2.train_all((np.array([[1, 2, 3], [3, 4, 5]]), np.array([9, 10])))
m(10, 20, 30)
mc2.print('A new MC, after wrapping a model and calling fit and predict', full=True)

After adding a model: call_target:host, #functions:0, model-qualities:[True], #tdata:0, #edata:0; tinputs:[]; toutputs:[]; einputs:[]; eoutputs:[]
A new MC, after wrapping a model and calling fit and predict: auto_cache=True, auto_id=True, auto_eval=True, auto_train=True, edata_fraction=0.3, feedback_fraction=0.1, qlty_threshold=0.95, ncargs=1, call_target:MC, #functions:0, model-qualities:[True], #tdata:2, #edata:1; tinputs:[[1, 2, 3, 2622603606032], [3, 4, 5, 2622603606032]]; toutputs:[9, 10]; einputs:[[10, 20, 30, 2622603606032]]; eoutputs:[18.0]
